# Tabelas de Medalhas dos Jogos Olímpicos de Verão, 1896-2014


## Passo 1: Preparação dos Dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Carregar os dados
summer = pd.read_csv("summer.csv")
wik_1996 = pd.read_csv("wik_1996.csv")
wik_1976 = pd.read_csv("wik_1976.csv")

In [3]:
# Inspecionar os dados
print("Primeiras linhas do arquivo summer.csv:")
print(summer.head())
print("\nÚltimas linhas do arquivo summer.csv:")
print(summer.tail())
print("\nLinhas com dados faltantes no arquivo summer.csv:")
print(summer[summer.isna().any(axis=1)])


Primeiras linhas do arquivo summer.csv:
   Year    City     Sport Discipline             Athlete Country Gender  \
0  1896  Athens  Aquatics   Swimming       HAJOS, Alfred     HUN    Men   
1  1896  Athens  Aquatics   Swimming    HERSCHMANN, Otto     AUT    Men   
2  1896  Athens  Aquatics   Swimming   DRIVAS, Dimitrios     GRE    Men   
3  1896  Athens  Aquatics   Swimming  MALOKINIS, Ioannis     GRE    Men   
4  1896  Athens  Aquatics   Swimming  CHASAPIS, Spiridon     GRE    Men   

                        Event   Medal  
0              100M Freestyle    Gold  
1              100M Freestyle  Silver  
2  100M Freestyle For Sailors  Bronze  
3  100M Freestyle For Sailors    Gold  
4  100M Freestyle For Sailors  Silver  

Últimas linhas do arquivo summer.csv:
       Year    City      Sport           Discipline  \
31160  2012  London  Wrestling  Wrestling Freestyle   
31161  2012  London  Wrestling  Wrestling Freestyle   
31162  2012  London  Wrestling  Wrestling Freestyle   
31163  201

In [4]:
# Remover dados faltantes
summer.dropna(inplace=True)

In [5]:
# Alinhar as tabelas oficiais com os dados
wik_1976["Country"] = wik_1976.NOC.str.split("(", expand=True).iloc[:, 1].str.replace(")", "").str.replace("*", "")
wik_1976 = wik_1976.drop(columns=["Rank", "NOC", "Total"]).set_index("Country")

wik_1996["Country"] = wik_1996.Nation.str.split("(", expand=True).iloc[:, 1].str.replace(")", "").str.replace("*", "")
wik_1996 = wik_1996.drop(columns=["Rank", "Nation", "Total"]).set_index("Country")

O primeiro passo prepara os dados para análise. As tabelas de medalhas oficiais são carregadas e alinhadas, ajustando os nomes dos países e removendo colunas desnecessárias. Além disso, as linhas com dados faltantes no arquivo `summer.csv` são removidas para garantir a integridade dos dados.

## Passo 2: Criação da Coluna Event_Gender

In [6]:
# Identificar eventos mistos
summer["Event_Gender"] = summer.Gender
mask1 = summer.Event.str.lower().str.contains("mixed")
mask2 = summer.Event.str.lower().str.contains("pairs")
mask3 = summer.Sport.str.lower().str.contains("equestrian")
mask4 = ((summer.Sport.str.lower().str.contains("sailing")) & (summer.Year < 1988))

summer.loc[mask1 | mask2 | mask3 | mask4, "Event_Gender"] = "X"


In [7]:
# Medals em Badminton misto
badm_mixed = [21773, 21782, 21776,21785, 21770, 21779,23703,23712,23706, 23715,23709,23700,25720,25729,25723,25732,25726,
              25717,27727,27736, 27730,27739,27724,27733, 29784, 29785,29786,29787,29788,29789]
summer.loc[badm_mixed, "Event_Gender"] = "X"

Neste passo, a coluna Event_Gender é criada para identificar se o evento é masculino, feminino ou misto. Regras específicas são aplicadas para identificar eventos mistos, e uma lista de eventos de Badminton é manualmente marcada como mista.

In [8]:
# Passo 3: Contagem de Medalhas por Evento
summer["Event_Medals"] = summer.groupby(["Year", "Sport", "Discipline", "Event", "Event_Gender"]).Medal.transform("count")

Este passo conta o número de medalhas atribuídas em cada evento, criando a coluna Event_Medals. Isso ajuda a identificar quantas medalhas foram concedidas em cada evento específico.


## Passo 4: Identificação de Eventos em Equipe

In [9]:
summer["Team"] = pd.Series(np.where(summer.Event_Medals > 5, "Yes", "No"))

Eventos com mais de 5 medalhas são considerados eventos de equipe e marcados na coluna Team. Isso permite diferenciar entre eventos individuais e eventos de equipe.

## Passo 5: Remoção de Medalhas Duplicadas em Eventos de Equipe

In [10]:
summer.reset_index(inplace=True)
singles = summer.loc[summer.Team == "No"].copy()
team = summer.loc[summer.Team == "Yes"].copy()
team.drop_duplicates(subset=["Year", "Sport", "Discipline", "Country", "Event", "Event_Gender", "Medal"], inplace=True)
summer_new = pd.concat([singles, team])
summer_new.set_index("index", inplace=True)

Este passo remove duplicatas de medalhas em eventos de equipe, mantendo apenas uma medalha por evento, país e tipo de medalha. Os dados são divididos em eventos individuais e de equipe, removendo duplicatas apenas dos eventos de equipe.


## Passo 6: Criação da Tabela Oficial de Medalhas para Todas as Edições

In [11]:
medal_tables = summer_new.groupby(["Year", "Country", "Medal"]).Medal.count().unstack(fill_value=0)[["Gold", "Silver", "Bronze"]]

As tabelas de medalhas para cada edição dos Jogos Olímpicos são criadas, contando o número de medalhas de ouro, prata e bronze por ano e país. Isso resulta em uma tabela agregada de medalhas para cada edição dos jogos.

## Passo 7: Comparação com as Tabelas de Medalhas da Wikipedia

In [12]:
agg_1976 = medal_tables.loc[1976].sort_values(["Gold", "Silver", "Bronze"], ascending=False).copy()
div_76 = agg_1976.sub(wik_1976).abs().dropna()
score_76 = div_76.sum().sum()

agg_1996 = medal_tables.loc[1996].sort_values(["Gold", "Silver", "Bronze"], ascending=False).copy()
div_96 = agg_1996.sub(wik_1996).abs().dropna()
score_96 = div_96.sum().sum()

print("Divergência total para a edição de 1976:", score_76)
print("Divergência total para a edição de 1996:", score_96)

Divergência total para a edição de 1976: 0.0
Divergência total para a edição de 1996: 0.0


As tabelas de medalhas produzidas para as edições de 1976 e 1996 são comparadas com as tabelas oficiais. A divergência total é calculada como a soma das diferenças absolutas entre o número de medalhas agregadas e o número oficial de medalhas. Os valores de score_76 e score_96 representam a divergência total para as edições de 1976 e 1996, respectivamente. O objetivo é minimizar esses valores, idealmente alcançando 0, o que indicaria que as tabelas produzidas são idênticas às tabelas oficiais.